# Region of Interest Analysis for TEMPEST

This notebook demonstrates how to select and analyze facets based on latitude/longitude coordinates.

**Example:** Bennu OSIRIS-REx candidate landing sites

## Key Features
- Select single facet closest to a lat/lon point
- Select all facets within a radius of a lat/lon point  
- Visualize selections using TEMPEST's existing animation viewer
- Calculate T^4 mean temperatures for selected regions
- Plot diurnal temperature curves vs local time


In [104]:
# Imports
import sys
import os
sys.path.insert(0, '..')

import numpy as np

from src.utilities.config import Config
from src.model.simulation import Simulation
from tempest import read_shape_model

# Import spatial selection functions
# NOTE: If you get ImportError, restart the kernel and run again
from src.utilities.spatial_selection import (
    latlon_to_cartesian,
    cartesian_to_latlon,
    find_closest_facet,
    find_facets_in_radius,
    calculate_shape_model_center
)
from src.utilities.plotting.animate_model import animate_model

print("✓ Imports successful")
print(f"✓ calculate_shape_model_center available: {callable(calculate_shape_model_center)}")


✓ Imports successful
✓ calculate_shape_model_center available: True


**Important:** If you get an ImportError above, you need to restart the kernel:
- In Jupyter: Kernel → Restart Kernel
- Then run all cells again from the beginning


## Setup: Change to TEMPEST root directory


In [105]:
# Change to TEMPEST root directory so file paths work correctly
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')
print(f"Working directory: {os.getcwd()}")


Working directory: /Users/duncan/Desktop/DPhil/TEMPEST


## 1. Load Shape Model


In [106]:
# Load configuration
config_file = "private/data/config/bennu/bennu_config.yaml"
config = Config(config_file)
simulation = Simulation(config)

# Load shape model
shape_model = read_shape_model(
    config.path_to_shape_model_file,
    simulation.timesteps_per_day,
    config.n_layers,
    config.max_days,
    config.calculate_energy_terms
)

print(f"Shape model loaded: {len(shape_model)} facets")

# Calculate mean radius
radii = [np.linalg.norm(f.position) for f in shape_model]
mean_radius = np.mean(radii)
print(f"Mean radius: {mean_radius:.2f} m")


Shape model loaded: 1966 facets
Mean radius: 245.15 m


### Check Shape Model Center

Verify that the shape model is centered at the origin. If not, we need to account for the offset when converting lat/lon coordinates.


In [107]:
# Calculate center of mass of the shape model
center_of_mass = calculate_shape_model_center(shape_model)
offset_magnitude = np.linalg.norm(center_of_mass)

print("Shape Model Center Analysis:")
print(f"  Center of mass: [{center_of_mass[0]:7.3f}, {center_of_mass[1]:7.3f}, {center_of_mass[2]:7.3f}] m")
print(f"  Offset from origin: {offset_magnitude:.3f} m")
print(f"  Offset as % of radius: {100 * offset_magnitude / mean_radius:.2f}%")

if offset_magnitude > 1.0:
    print(f"\nWARNING: Shape model center is {offset_magnitude:.1f}m from origin!")
    print("   This will cause systematic errors in lat/lon conversions.")
    print("   The functions will automatically account for this offset.")
else:
    print("\n✓ Shape model is well-centered at origin")


Shape Model Center Analysis:
  Center of mass: [ -1.712,  -2.259,  -3.896] m
  Offset from origin: 4.818 m
  Offset as % of radius: 1.97%

   This will cause systematic errors in lat/lon conversions.
   The functions will automatically account for this offset.


## 2. Test Coordinate System

Quick test to verify lat/lon ↔ Cartesian conversions work correctly.


In [108]:
# Test coordinate conversion
test_points = [
    (90, 0, "North Pole"),
    (-90, 0, "South Pole"),
    (0, 0, "Equator at 0°E"),
    (0, 90, "Equator at 90°E"),
]

radius = 245.0  # Bennu radius

print("Coordinate System Convention:")
print("  Z-axis = North Pole (+90°)")
print("  X-axis = 0° longitude")
print("  Y-axis = 90° longitude")
print("\nCoordinate Test:")
print("=" * 70)

for lat, lon, desc in test_points:
    xyz = latlon_to_cartesian(lat, lon, radius)
    lat_back, lon_back = cartesian_to_latlon(xyz)
    print(f"{desc:20s} ({lat:4.0f}°N, {lon:4.0f}°E)")
    print(f"  → XYZ: [{xyz[0]:7.1f}, {xyz[1]:7.1f}, {xyz[2]:7.1f}] m")
    print(f"  → Back: ({lat_back:4.0f}°N, {lon_back:4.0f}°E)")
    print()


Coordinate System Convention:
  Z-axis = North Pole (+90°)
  X-axis = 0° longitude
  Y-axis = 90° longitude

Coordinate Test:
North Pole           (  90°N,    0°E)
  → XYZ: [    0.0,     0.0,   245.0] m
  → Back: (  90°N,    0°E)

South Pole           ( -90°N,    0°E)
  → XYZ: [    0.0,     0.0,  -245.0] m
  → Back: ( -90°N,    0°E)

Equator at 0°E       (   0°N,    0°E)
  → XYZ: [  245.0,     0.0,     0.0] m
  → Back: (   0°N,    0°E)

Equator at 90°E      (   0°N,   90°E)
  → XYZ: [    0.0,   245.0,     0.0] m
  → Back: (   0°N,   90°E)



## 3. Define Region/s of Interest

Modify these coordinates for your own body/sites.


In [109]:
LANDING_SITES = {
    'Nightingale': {
        'lat': 56.0,
        'lon': 43.0,
        'description': 'Primary landing site - selected for sample collection'
    },
    'Osprey': {
        'lat': 11.0,
        'lon': 88.0,
        'description': 'Backup landing site'
    },
    'Kingfisher': {
        'lat': 11.0,
        'lon': 56.0,
        'description': 'Candidate landing site'
    },
    'Sandpiper': {
        'lat': -47.0,
        'lon': 322.0,
        'description': 'Candidate landing site'
    }
}

BENNU_RADIUS = 245.0  # meters - mean radius
SITE_RADIUS = 25.0    # meters - search radius for circular region

print("Landing sites defined:")
for name, info in LANDING_SITES.items():
    print(f"  {name}: {info['lat']}°N, {info['lon']}°E")


Landing sites defined:
  Nightingale: 56.0°N, 43.0°E
  Osprey: 11.0°N, 88.0°E
  Kingfisher: 11.0°N, 56.0°E
  Sandpiper: -47.0°N, 322.0°E


## 4. Select Facets for Each Region of Interst

Two selection methods:
1. **`find_closest_facet()`** - Always returns THE single closest facet
2. **`find_facets_in_radius()`** - Returns ALL facets within a circular region (can return 0 if none nearby)


In [110]:
site_selections = {}

print("Region of Interest Facet Selection")
print("=" * 80)

for site_name, site_info in LANDING_SITES.items():
    lat = site_info['lat']
    lon = site_info['lon']
    
    # Method 1: Find closest facet (always returns 1)
    closest_id, closest_dist = find_closest_facet(shape_model, lat, lon, BENNU_RADIUS, center_of_mass)
    closest_pos = shape_model[closest_id].position
    closest_lat, closest_lon = cartesian_to_latlon(closest_pos, center_of_mass)
    
    # Method 2: Find all facets within radius (can return 0)
    facets_in_radius = find_facets_in_radius(shape_model, lat, lon, BENNU_RADIUS, SITE_RADIUS, center_of_mass)
    
    # Store results
    site_selections[site_name] = {
        'target_lat': lat,
        'target_lon': lon,
        'closest_facet_id': closest_id,
        'closest_distance': closest_dist,
        'facets_in_radius': facets_in_radius,
        'description': site_info['description']
    }
    
    # Print summary
    print(f"\n{site_name}:")
    print(f"  Description: {site_info['description']}")
    print(f"  Target: {lat}°N, {lon}°E")
    print(f"  Closest facet: {closest_id}")
    print(f"  Closest position: {closest_lat:.1f}°N, {closest_lon:.1f}°E")
    print(f"  Distance to closest: {closest_dist:.2f} m")
    print(f"  Facets within {SITE_RADIUS}m radius: {len(facets_in_radius)}")
    
    if facets_in_radius:
        total_area = sum(shape_model[fid].area for fid in facets_in_radius)
        print(f"  Total area of region: {total_area:.2f} m²")
    elif closest_dist > SITE_RADIUS:
        print(f"No facets within radius (increase SITE_RADIUS to >{closest_dist:.0f}m to include closest)")

print("\n" + "=" * 80)


Region of Interest Facet Selection

Nightingale:
  Description: Primary landing site - selected for sample collection
  Target: 56.0°N, 43.0°E
  Closest facet: 237
  Closest position: 55.1°N, 44.8°E
  Distance to closest: 23.38 m
  Facets within 25.0m radius: 6
  Total area of region: 1801.31 m²

Osprey:
  Description: Backup landing site
  Target: 11.0°N, 88.0°E
  Closest facet: 1176
  Closest position: 12.8°N, 87.0°E
  Distance to closest: 19.82 m
  Facets within 25.0m radius: 4
  Total area of region: 2161.79 m²

Kingfisher:
  Description: Candidate landing site
  Target: 11.0°N, 56.0°E
  Closest facet: 1064
  Closest position: 9.9°N, 55.5°E
  Distance to closest: 20.61 m
  Facets within 25.0m radius: 4
  Total area of region: 1665.74 m²

Sandpiper:
  Description: Candidate landing site
  Target: -47.0°N, 322.0°E
  Closest facet: 1943
  Closest position: -45.9°N, 319.6°E
  Distance to closest: 20.37 m
  Facets within 25.0m radius: 4
  Total area of region: 1785.92 m²



In [111]:
print("ROI Facet IDs - Detailed List")
print("=" * 80)

for site_name, data in site_selections.items():
    print(f"\n{site_name}:")
    print(f"  Target: {data['target_lat']}°N, {data['target_lon']}°E")
    print(f"  Description: {data['description']}")
    print(f"  Closest facet: {data['closest_facet_id']}")
    
    facets = data['facets_in_radius']
    print(f"  Number of facets in radius: {len(facets)}")
    
    if facets:
        total_area = sum(shape_model[fid].area for fid in facets)
        print(f"  Total area: {total_area:.2f} m²")
        print(f"  Expected area (π×25²): {np.pi * 25**2:.2f} m²")
        
        # Print facet IDs in a nice format
        print(f"\n  All facet IDs ({len(facets)} total):")
        # Print 10 IDs per line
        for i in range(0, len(facets), 10):
            batch = facets[i:i+10]
            id_str = ", ".join(f"{fid:5d}" for fid in batch)
            print(f"    {id_str}")
    else:
        print("  No facets found within radius!")
    
print("\n" + "=" * 80)


ROI Facet IDs - Detailed List

Nightingale:
  Target: 56.0°N, 43.0°E
  Description: Primary landing site - selected for sample collection
  Closest facet: 237
  Number of facets in radius: 6
  Total area: 1801.31 m²
  Expected area (π×25²): 1963.50 m²

  All facet IDs (6 total):
      237,   243,   212,   255,   261,   226

Osprey:
  Target: 11.0°N, 88.0°E
  Description: Backup landing site
  Closest facet: 1176
  Number of facets in radius: 4
  Total area: 2161.79 m²
  Expected area (π×25²): 1963.50 m²

  All facet IDs (4 total):
     1176,  1175,  1166,  1194

Kingfisher:
  Target: 11.0°N, 56.0°E
  Description: Candidate landing site
  Closest facet: 1064
  Number of facets in radius: 4
  Total area: 1665.74 m²
  Expected area (π×25²): 1963.50 m²

  All facet IDs (4 total):
     1064,  1065,  1053,  1076

Sandpiper:
  Target: -47.0°N, 322.0°E
  Description: Candidate landing site
  Closest facet: 1943
  Number of facets in radius: 4
  Total area: 1785.92 m²
  Expected area (π×25²): 1